# Install and Import Dependencies

In [ ]:
!pip install numpy pandas librosa groq load_dotenv

In [ ]:
!pip install nltk tiktoken parselmouth pydub psutil

In [ ]:
# Imports
import io
import os
import re
import joblib
import asyncio
import librosa
import tiktoken
import numpy as np
import parselmouth
from pydub import AudioSegment
from nltk.corpus import cmudict
from parselmouth.praat import call
from groq import Groq, AsyncClient
from groq.types.audio import Transcription

# Load environment file
from load_dotenv import load_dotenv
print(load_dotenv('../../.env.local'))

assert os.environ.get('GROQ_API_KEY'), "Groq API key not found in .env file, please set the key before starting this notebook"

# Global variables
client = AsyncClient()
encoder = tiktoken.get_encoding('gpt2')
fluency_model = joblib.load('../fluency/weights/xgboost_model.pkl')

try:
    cmu_dict = cmudict.dict()
except:
    import nltk
    nltk.download('cmudict')
    cmu_dict = cmudict.dict()    

## Monitor CPU resources

In [ ]:
import psutil
import time
import functools

def monitor_resources(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        process = psutil.Process(os.getpid())

        # Get memory and CPU before
        mem_before = process.memory_info().rss / (1024 ** 2)  # MB
        cpu_before = process.cpu_percent(interval=None)

        # Start time and CPU
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()

        # Get memory and CPU after
        mem_after = process.memory_info().rss / (1024 ** 2)  # MB
        cpu_after = process.cpu_percent(interval=0.1)

        # Get number of CPUs used
        cpu_affinity = process.cpu_affinity()
        
        print(f"Function: {func.__name__}")
        print(f"Execution Time: {end_time - start_time:.2f} sec")
        print(f"Memory Usage: {mem_after - mem_before:.2f} MB")
        print(f"CPU Usage: {cpu_after:.2f}%")
        print(f"CPU Cores Used: {cpu_affinity}")

        return result
    return wrapper

def limit_to_one_core(core_id=0):
    """
    Set process to run only on one CPU core (default: core 0).
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            process = psutil.Process(os.getpid())

            # Store the current affinity to restore later
            original_affinity = process.cpu_affinity()
            
            try:
                # Set affinity to a single core
                process.cpu_affinity([core_id])
                print(f"Running {func.__name__} on CPU core {core_id}")
                return func(*args, **kwargs)
            finally:
                # Restore original affinity
                process.cpu_affinity(original_affinity)
        return wrapper
    return decorator

# # Limit NumPy, OpenBLAS etc to use only one CPU core
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["OPENBLAS_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
# os.environ["NUMEXPR_NUM_THREADS"] = "1"

# Feature Extraction
Features extracted:
* ZCR
* Pitch
* Jitter
* Shimmer
* Harmonic-to-Noise ratio
* RMS
* MFCC
* DeltaMFCC
* SpeakingRate
* PauseCount
* PauseDuration

In [ ]:
# Async Transcription
def split_audio_in_memory(audio_path, max_mb=24):
    audio = AudioSegment.from_wav(audio_path)
    bytes_per_ms = (audio.frame_rate * audio.frame_width * audio.channels) / 1000
    max_bytes = max_mb * 1024 * 1024
    chunk_duration_ms = int(max_bytes / bytes_per_ms)

    chunks = []
    for i in range(0, len(audio), chunk_duration_ms):
        chunk = audio[i:i+chunk_duration_ms]
        buffer = io.BytesIO()
        chunk.export(buffer, format="wav")
        buffer.seek(0)
        chunks.append((f"chunk_{i//chunk_duration_ms}.wav", buffer))

    return chunks

async def transcribe_chunk(filename, audio_buffer):
    return await client.audio.transcriptions.create(
        file=(filename, audio_buffer.read()),
        model="distil-whisper-large-v3-en",
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )


async def transcribe_audio(audio_path, client=client):
    """Transcribe an audio file without saving the chunks to disk"""
    chunks = split_audio_in_memory(audio_path)
    tasks = [transcribe_chunk(name, buffer) for name, buffer in chunks]
    all_transcripts = await asyncio.gather(*tasks)

    transcript_parts = []
    all_words = []
    total_duration = 0.0

    for chunk in all_transcripts:
        transcript_parts.append(chunk.text)
        all_words.extend(getattr(chunk, "words", []))
        total_duration += chunk.duration

    transcript = "".join(transcript_parts)
    
    return Transcription(text=transcript, words=all_words, duration=total_duration)


transcript = await transcribe_audio("../../samples/confident.wav")
transcript

In [ ]:
# Helper functions for calculating syllables speaking rate
def get_word_syllable_count(word):
    word = word.lower().strip(".,?!;:")
    if word in cmu_dict:
        return len([p for p in cmu_dict[word][0] if p[-1].isdigit()])
    return max(1, len(re.findall(r'[aeiouy]+', word)))


def estimate_syllable_rate(transcript, duration_sec):
    words = transcript.split()
    total_syllables = sum(get_word_syllable_count(word) for word in words)
    return total_syllables / duration_sec if duration_sec > 0 else 0

In [ ]:
# Extract Pitch statistics, Jitter, Shimmer, and HNR ratio through Parselmouth
@monitor_resources
def extract_parselmouth_features(data, sr):
    snd = parselmouth.Sound(values=data, sampling_frequency=sr)

    pitch_obj = snd.to_pitch()
    pitch_mean = call(pitch_obj, "Get mean", 0, 0, "Hertz")
    pitch_std = call(pitch_obj, "Get standard deviation", 0, 0, "Hertz")

    point_process = call(snd, "To PointProcess (periodic, cc)", 75, 500)
    jitter = call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    shimmer = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    harmonicity = call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)

    return {
        "pitch_mean": pitch_mean,
        "pitch_std": pitch_std,
        "pitch_var": pitch_std**2,
        "jitter_local": jitter,
        "shimmer_local": shimmer,
        "hnr": hnr
    }

async def async_extract_parselmouth_features(data, sr, executor):
    return await asyncio.get_event_loop().run_in_executor(
        executor, extract_parselmouth_features, data, sr
    )

In [ ]:
# Extract RMS Energy, ZCR, MFCC and Deltas using librosa
@monitor_resources
def extract_librosa_features(data, sr):
    zcr = np.mean(librosa.feature.zero_crossing_rate(data))
    
    rms = librosa.feature.rms(y=data)[0]
    rms_mean = np.mean(rms)
    rms_std = np.std(rms)
    rms_var = np.var(rms)

    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=13)
    delta = librosa.feature.delta(mfcc)
    mfcc_mean = np.mean(mfcc)
    delta_mean = np.mean(delta)

    return {
        "zcr": zcr,
        "rms_mean": rms_mean,
        "rms_std": rms_std,
        "rms_var": rms_var,
        "mfcc": mfcc.mean(axis=1),
        "delta_mfcc": delta.mean(axis=1),
        "mfcc_mean": mfcc_mean,
        "delta_mean": delta_mean
    }
    
async def async_extract_librosa_features(data, sr, executor):
    return await asyncio.get_event_loop().run_in_executor(
        executor, extract_librosa_features, data, sr
    )

In [ ]:
def extract_features_from_wave(data, sr):
    return {
        **extract_librosa_features(data, sr),
        **extract_parselmouth_features(data, sr)
    }
    
async def async_extract_features_from_wave(data, sr, executor):
    # Start both tasks concurrently
    librosa_task = asyncio.create_task(async_extract_librosa_features(data, sr, executor))
    parselmouth_task = asyncio.create_task(async_extract_parselmouth_features(data, sr, executor))

    # Wait for both
    librosa_feats, parselmouth_feats = await asyncio.gather(librosa_task, parselmouth_task)

    return {**librosa_feats, **parselmouth_feats}


@monitor_resources
async def extract_features(audio_path, baseline_duration: float = 0.0, fluency_model=fluency_model):
    # -------------- Load the audio file --------------
    data, sr = librosa.load(audio_path)
    assert len(data) != 0, "Your audio file appears to contain no content. Please input a valid file"
    
    
    # -------------- Get transcription and check minimum duration --------------
    transcription_json = await transcribe_audio(audio_path)
    duration_sec = transcription_json.duration # type: ignore
    baseline_duration = baseline_duration or max(10.0, duration_sec * 0.05)

    assert duration_sec != 0, "File duration appears to be 0 after transcription?"
    
    
    # -------------- Get features of baseline and full wave --------------
    baseline_data = data[:min(len(data), int(sr * baseline_duration))]
    baseline_feats = extract_features_from_wave(baseline_data, sr)
    full_feats = extract_features_from_wave(data, sr)


    # -------------- Get fluency ratings --------------
    features = ['zcr', 'pitch_mean', 'pitch_std', 'rms_mean', 'rms_std', 'rms_var', 'mfcc_mean', 'delta_mean']
    rating_map = ['Low', 'Medium', 'High']
        
    baseline_fluency_features = np.array([baseline_feats[key] for key in baseline_feats if key in features])
    full_fluency_features = np.array([full_feats[key] for key in full_feats if key in features])

    res = fluency_model.predict(np.vstack((baseline_fluency_features, full_fluency_features)))
    baseline_fluency = rating_map[res[0].argmax()]
    full_fluency = rating_map[res[1].argmax()]

    relative_feats = {}
    for key in full_feats:
        if key not in ['mfcc', 'delta_mfcc']:
            base = baseline_feats.get(key, 0.0)
            full = full_feats[key]
            relative_feats[f'{key}_delta'] = full - base
    
    
    # -------------- Get speaking rates --------------
    # Assuming the transcript has come by now

    # Baseline speaking rate
    baseline_transcript = [word_segment['word'] for word_segment in transcription_json.words if word_segment['start'] <= baseline_duration]  # type: ignore
    baseline_word_count = len(baseline_transcript)
    baseline_transcript = " ".join(baseline_transcript)
    baseline_speaking_rate = baseline_word_count / baseline_duration
    baseline_syllables_rate = estimate_syllable_rate(baseline_transcript, baseline_duration)

    # Full data speaking rate
    transcript = transcription_json.text
    word_count = len(transcript.split())
    speaking_rate = word_count / duration_sec
    syllables_rate = estimate_syllable_rate(transcript, duration_sec)
        
    
    # -------------- Pause detection --------------
    intervals = librosa.effects.split(data, top_db=30)
    pauses = [(intervals[i][0] - intervals[i - 1][1]) / sr
              for i in range(1, len(intervals))
              if (intervals[i][0] - intervals[i - 1][1]) / sr > 1.0]
    
    long_pause_count = len(pauses)
    long_pause_total = sum(pauses)

    return {
        "transcript": transcript,
        "duration": duration_sec,
        "baseline_duration": baseline_duration,
        "speaking_rate": speaking_rate,
        "syllables_rate": syllables_rate,
        "baseline_speaking_rate": baseline_speaking_rate,
        "baseline_syllables_rate": baseline_syllables_rate,
        "long_pause_count": long_pause_count,
        "long_pause_duration": long_pause_total,
        "fluency_rating": full_fluency,
        "baseline_fluency_rating": baseline_fluency,
        **full_feats,
        **{f'baseline_{k}': v for k, v in baseline_feats.items()},
        **relative_feats,
    }


In [ ]:
features = await extract_features('../../samples/tim-urban.wav')

In [ ]:
features

# Send to GPT for feedback

In [ ]:
def get_prompt(audio_features, posture_features = None):
    prompt = f"""
You are a professional voice coach and delivery analyst tasked with evaluating a speaker's performance based on a variety of acoustic and prosodic features. Below is a detailed snapshot of the speaker’s delivery — both baseline and full-clip — along with their changes. Use this to deliver personalized, context-aware feedback.

## NOTE:
- The **first {int(audio_features['baseline_duration'])} seconds** of the speech are used to define the speaker's personal baseline.
- All relative metrics (e.g., deltas, ratios) are calculated with respect to this baseline.
- Interpret *changes* from baseline as signs of adaptation or stress — not necessarily flaws.
- **Avoid quoting any raw values** in your response. Use intuitive, narrative insights only.
- An 86% accurate ML model was used to rate the fluency of the speech, and that rating has also been provided to you.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 TRANSCRIPT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
<transcript>
{audio_features['transcript']}
</transcript>

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📏 BASELINE METRICS (First {int(audio_features['baseline_duration'])} seconds)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Fluency & Tempo
- Fluency rating: {audio_features['baseline_fluency_rating']}
- Words/sec: {audio_features['baseline_speaking_rate']:.2f}
- Syllables/sec: {audio_features['baseline_syllables_rate']:.2f}

## Voice Modulation
- Pitch (Mean / Std / Var): {audio_features['baseline_pitch_mean']:.2f} / {audio_features['baseline_pitch_std']:.2f} / {audio_features['baseline_pitch_var']:.2f}
- Jitter (local): {audio_features['baseline_jitter_local']:.3f}
- Shimmer (local): {audio_features['baseline_shimmer_local']:.3f}
- Harmonic-to-Noise Ratio (HNR): {audio_features['baseline_hnr']:.2f}

## Energy & Dynamics
- RMS Energy (Mean / Std / Var): {audio_features['baseline_rms_mean']:.2f} / {audio_features['baseline_rms_std']:.2f} / {audio_features['baseline_rms_var']:.2f}
- Zero Crossing Rate: {audio_features['baseline_zcr']:.3f}

## Timbre & Articulation
- MFCC Mean: {audio_features['baseline_mfcc_mean']:.2f}
- Delta MFCC Mean: {audio_features['baseline_delta_mean']:.6f}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 FULL CLIP METRICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Fluency & Tempo
- Fluency rating: {audio_features['fluency_rating']}
- Words/sec: {audio_features['speaking_rate']:.2f}
- Syllables/sec: {audio_features['syllables_rate']:.2f}
- Long pauses (>1s): {audio_features['long_pause_count']}
- Total pause duration: {audio_features['long_pause_duration']:.2f} sec

## Voice Modulation
- Pitch (Mean / Std / Var): {audio_features['pitch_mean']:.2f} / {audio_features['pitch_std']:.2f} / {audio_features['pitch_var']:.2f}
- Jitter (local): {audio_features['jitter_local']:.3f}
- Shimmer (local): {audio_features['shimmer_local']:.3f}
- Harmonic-to-Noise Ratio (HNR): {audio_features['hnr']:.2f}

## Energy & Dynamics
- RMS Energy (Mean / Std / Var): {audio_features['rms_mean']:.2f} / {audio_features['rms_std']:.2f} / {audio_features['rms_var']:.2f}
- Zero Crossing Rate: {audio_features['zcr']:.3f}

## Timbre & Articulation
- MFCC Mean: {audio_features['mfcc_mean']:.2f}
- Delta MFCC Mean: {audio_features['delta_mean']:.6f}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📈 RELATIVE CHANGES FROM BASELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Tempo & Fluency
- Speaking rate ratio: {audio_features['speaking_rate'] / audio_features['baseline_speaking_rate']:.2f}
- Syllable rate ratio: {audio_features['syllables_rate'] / audio_features['baseline_syllables_rate']:.2f}

## Modulation
- Pitch std delta: {audio_features['pitch_std_delta']:+.2f}
- Jitter delta: {audio_features['jitter_local_delta']:+.3f}
- Shimmer delta: {audio_features['shimmer_local_delta']:+.3f}
- HNR delta: {audio_features['hnr_delta']:+.2f}

## Energy
- RMS mean delta: {audio_features['rms_mean_delta']:+.2f}
- RMS std delta: {audio_features['rms_std_delta']:+.2f}
- ZCR delta: {audio_features['zcr_delta']:+.3f}

## Timbre
- MFCC mean delta: {audio_features['mfcc_mean_delta']:+.2f}
- Delta MFCC mean delta: {audio_features['delta_mean_delta']:+.6f}

🧠 **Interpretation Tips** (for internal use only):
- A **negative pitch_std_delta** might suggest monotony or nervousness; a positive value implies expressive modulation.
- **Decreased RMS or HNR** may imply loss of vocal energy or confidence.
- **Increased jitter/shimmer** may reflect stress or instability.
- A **low syllable rate ratio** suggests slowing down relative to their natural pace, which may imply hesitation or deliberate pacing.
- **ZCR changes** may reflect articulation style or clarity.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🧭 INSTRUCTIONS FOR FEEDBACK GENERATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Using the data above, write a highly personalized and supportive **narrative-style voice coaching paragraph**. Do not cite any specific numerical values. Your tone should be professional, encouraging, and practical.

Structure your feedback in **three sections**:

1. ✅ **What the speaker did well** — Highlight strengths or improvements in vocal control, energy, fluency, or confidence.
2. 🛠️ **What they can improve** — Tactfully mention areas that deviated from their baseline and might affect clarity or delivery.
3. 📊 **Confidence & fluency rating** — Conclude with your overall impression of their vocal confidence and fluency (e.g., low, moderate, high), based on relative metrics.

DO NOT compare to average speakers. DO NOT be generic. Focus only on deviations from this speaker's own baseline and the emotional/functional impact of those changes.
"""
    return prompt

In [ ]:
from schema import AudioOnlyFeedback, FeedbackSchema

def get_prompt_with_schema(audio_features, posture_features = None, schema = None):
    prompt = f"""
You are a professional voice coach and delivery analyst tasked with evaluating a speaker's performance based on a variety of acoustic and prosodic features. Below is a detailed snapshot of the speaker’s delivery — both baseline and full-clip — along with their changes. Use this to deliver personalized, context-aware feedback.

## NOTE:
- The **first {int(audio_features['baseline_duration'])} seconds** of the speech are used to define the speaker's personal baseline.
- All relative metrics (e.g., deltas, ratios) are calculated with respect to this baseline.
- Interpret *changes* from baseline as signs of adaptation or stress — not necessarily flaws.
- **Avoid quoting any raw values** in your response. Use intuitive, narrative insights only.
- An 86% accurate ML model was used to rate the fluency of the speech, and that rating has also been provided to you.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 TRANSCRIPT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
<transcript>
{audio_features['transcript']}
</transcript>

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📏 BASELINE METRICS (First {int(audio_features['baseline_duration'])} seconds)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Fluency & Tempo
- Fluency rating: {audio_features['baseline_fluency_rating']}
- Words/sec: {audio_features['baseline_speaking_rate']:.2f}
- Syllables/sec: {audio_features['baseline_syllables_rate']:.2f}

## Voice Modulation
- Pitch (Mean / Std / Var): {audio_features['baseline_pitch_mean']:.2f} / {audio_features['baseline_pitch_std']:.2f} / {audio_features['baseline_pitch_var']:.2f}
- Jitter (local): {audio_features['baseline_jitter_local']:.3f}
- Shimmer (local): {audio_features['baseline_shimmer_local']:.3f}
- Harmonic-to-Noise Ratio (HNR): {audio_features['baseline_hnr']:.2f}

## Energy & Dynamics
- RMS Energy (Mean / Std / Var): {audio_features['baseline_rms_mean']:.2f} / {audio_features['baseline_rms_std']:.2f} / {audio_features['baseline_rms_var']:.2f}
- Zero Crossing Rate: {audio_features['baseline_zcr']:.3f}

## Timbre & Articulation
- MFCC Mean: {audio_features['baseline_mfcc_mean']:.2f}
- Delta MFCC Mean: {audio_features['baseline_delta_mean']:.6f}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 FULL CLIP METRICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Fluency & Tempo
- Fluency rating: {audio_features['fluency_rating']}
- Words/sec: {audio_features['speaking_rate']:.2f}
- Syllables/sec: {audio_features['syllables_rate']:.2f}
- Long pauses (>1s): {audio_features['long_pause_count']}
- Total pause duration: {audio_features['long_pause_duration']:.2f} sec

## Voice Modulation
- Pitch (Mean / Std / Var): {audio_features['pitch_mean']:.2f} / {audio_features['pitch_std']:.2f} / {audio_features['pitch_var']:.2f}
- Jitter (local): {audio_features['jitter_local']:.3f}
- Shimmer (local): {audio_features['shimmer_local']:.3f}
- Harmonic-to-Noise Ratio (HNR): {audio_features['hnr']:.2f}

## Energy & Dynamics
- RMS Energy (Mean / Std / Var): {audio_features['rms_mean']:.2f} / {audio_features['rms_std']:.2f} / {audio_features['rms_var']:.2f}
- Zero Crossing Rate: {audio_features['zcr']:.3f}

## Timbre & Articulation
- MFCC Mean: {audio_features['mfcc_mean']:.2f}
- Delta MFCC Mean: {audio_features['delta_mean']:.6f}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📈 RELATIVE CHANGES FROM BASELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Tempo & Fluency
- Speaking rate ratio: {audio_features['speaking_rate'] / audio_features['baseline_speaking_rate']:.2f}
- Syllable rate ratio: {audio_features['syllables_rate'] / audio_features['baseline_syllables_rate']:.2f}

## Modulation
- Pitch std delta: {audio_features['pitch_std_delta']:+.2f}
- Jitter delta: {audio_features['jitter_local_delta']:+.3f}
- Shimmer delta: {audio_features['shimmer_local_delta']:+.3f}
- HNR delta: {audio_features['hnr_delta']:+.2f}

## Energy
- RMS mean delta: {audio_features['rms_mean_delta']:+.2f}
- RMS std delta: {audio_features['rms_std_delta']:+.2f}
- ZCR delta: {audio_features['zcr_delta']:+.3f}

## Timbre
- MFCC mean delta: {audio_features['mfcc_mean_delta']:+.2f}
- Delta MFCC mean delta: {audio_features['delta_mean_delta']:+.6f}

🧠 **Interpretation Tips** (for internal use only):
- A **negative pitch_std_delta** might suggest monotony or nervousness; a positive value implies expressive modulation.
- **Decreased RMS or HNR** may imply loss of vocal energy or confidence.
- **Increased jitter/shimmer** may reflect stress or instability.
- A **low syllable rate ratio** suggests slowing down relative to their natural pace, which may imply hesitation or deliberate pacing.
- **ZCR changes** may reflect articulation style or clarity.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🧭 INSTRUCTIONS FOR FEEDBACK GENERATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Using the data above, write a highly personalized and supportive **narrative-style voice coaching paragraph**. Do not cite any specific numerical values. Your tone should be professional, encouraging, and practical.

Structure your feedback in **three sections**:

1. ✅ **What the speaker did well** — Highlight strengths or improvements in vocal control, energy, fluency, or confidence.
2. 🛠️ **What they can improve** — Tactfully mention areas that deviated from their baseline and might affect clarity or delivery.
3. 📊 **Confidence & fluency rating** — Conclude with your overall impression of their vocal confidence and fluency (e.g., low, moderate, high), based on relative metrics.

DO NOT compare to average speakers. DO NOT be generic. Focus only on deviations from this speaker's own baseline and the emotional/functional impact of those changes.
"""
    return prompt


def generate_feedback(audio_features, posture_features = None, response_schema = None, llm_model : str = "llama-3.3-70b-versatile"):
    prompt = get_prompt(audio_features)

    client = Groq()
    completion = client.chat.completions.create(
        model=llm_model,
        messages=[
        {
            "role": "user",
            "content": prompt
        }
        ],
        temperature=0.5,
        max_completion_tokens=32768,
        top_p=1,
        response_format={"type": "json_object"},
        stream=False,
        stop=None,
    )

    return completion.choices[0].message

# Run Pipeline

In [ ]:
def get_n_tokens(features): return len(encoder.encode(get_prompt(features)))

In [ ]:
# An unconfident speech
path = "../../samples/unconfident.wav"
features = await extract_features(path)
feedback = generate_feedback(features)

get_n_tokens(features)

In [ ]:
import IPython.display as ipd
ipd.Markdown(feedback.content)

In [ ]:
# A confident speech
path = "../../samples/confident.wav"
features = await extract_features(path)
feedback = generate_feedback(features)

get_n_tokens(features)

In [ ]:
features

In [ ]:
ipd.Markdown(feedback.content)

In [ ]:
tim_urban_path = "../../samples/tim-urban.wav"
tim_urban_features = await extract_features(tim_urban_path)
tim_urban_feedback = generate_feedback(tim_urban_features)

In [ ]:
get_n_tokens(tim_urban_features)

In [ ]:
tim_urban_features

In [ ]:
import IPython.display as ipd
ipd.Markdown(tim_urban_feedback.content)